# **Lab 0. ROS2**


Teamwork: 2-3 people

## **The objectives of today's lab**

*  To learn or refresh odometry concept in ROS2



* Refresh how the ROS 2 environment is structured: packages, workspace, building with `colcon`.
* Refresh core ROS 2 concepts: **node**, **topic**, **message**, basic QoS.
* Implement a simple **publisher/subscriber** pair in Python (`rclpy`).
* Refresh the basics of incremental encoders: A/B quadrature, direction, counting modes (x1/x2/x4). TO-DO
* Understand how **odometry** is published in ROS 2: `nav_msgs/msg/Odometry` plus TF `odom → base_link`.

## **Pre-lab requirements (students should prepare before lab)**

[Only a machine with Docker installed](https://docs.docker.com/engine/install/)

[Don't forget to add your user to the docker group Linux](https://docs.docker.com/engine/install/linux-postinstall)

> *note:* The best way is to use native Ubuntu 22.04 or higher with Docker Engine

## **What you need to do**

1. Start the container and verify demo nodes and GUI (RViz2) work.
2. Implement a “fake encoder” node (publishes wheel tick counters).
3. Implement an “encoder driver” node (subscribes to ticks, publishes wheel angles via `/joint_states`).
4. Implement an odometry node (subscribes to wheel angles, publishes `/odom` and TF).
5. Write simple report

*More detailed description in following block*


## **Questions you will need to answer in the report**

1. Attach a screenshot of `listener` from Task 1 and briefly explain what you observe.
2. Attach a screenshots of `rqt_graph` and explain which nodes and topics are connected in your solution.
3. Briefly explains the math that goes into calculating your wheel rotation to odometry.
4. Why publish both `nav_msgs/Odometry` **and** TF `odom → base_link`? Which `frame_id` and `child_frame_id` do you use and why?


---

## Task 1, System

### 1.1 Container and workspace

We use Docker to ensure the same environment for everyone. Typical project structure:

* `Dockerfile` — defines the image (OS/ROS2/packages).
* `docker-compose.yaml` — defines services, environment variables, GUI forwarding, and mounted folders.

Key point is that the repo source directory is mounted into the container as `~/ros2_ws/src/`. Build artifacts (`build/`, `install/`, `log/`) and all other files are created inside the container.

To run container do the following in your terminal:

```bash
docker compose up <service>
```

List of services:
* terminal_linux - most common
* terminal_arm - for Apple Silicon, Raspberry, Jetson
* terminal_win11 - for Windows OS 

To attach to the running container
```bash
docker compose exec <service> bash
```

### 1.2 Demo publisher/subscriber

Open **two** terminals inside the container.


Terminal A:

Build workspace via:

```bash
colcon build
```

After sucessefull build add enivronment:
```bash
source install/setup.bash
```

And starts first node:
```bash
ros2 run sensors_and_sensing_demo talker
```

Terminal B:

You should add enivronment in all terminals:
```bash
source install/setup.bash
```

And finally start second node:
```bash
ros2 run sensors_and_sensing_demo listener
```

> Checkpoint (for the report): screenshot of the running `listener`.

### 1.3 Inspect the ROS 2 graph

In a third terminal:

```bash
ros2 node list
ros2 topic list
ros2 topic info /topic
ros2 topic echo /topic
```

Open the graph viewer:

```bash
ros2 run rqt_graph rqt_graph
```

`rqt_graph` shows nodes, topics, and their connections.


If you don't see a new window and the terminal displays an error, the image could not be forwarded from the container.

A classic Ubuntu problem is the security system; just enter in your **local machine**:
```bash
xhost +local:
```

If the problem is non-trivial... cry, try to figure it out yourself, or ask your teacher.

---

## Task 2, Fake encoder

### 2.1 Goal

[Follow the official instructions](https://docs.ros.org/en/humble/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Py-Publisher-And-Subscriber.html)

Implement a node `fake_encoder` that **does not read any hardware** and periodically publishes left and right wheel encoder ticks (counters).

Recommended message type: [`std_msgs/msg/Int32MultiArray`](https://docs.ros.org/en/noetic/api/std_msgs/html/msg/Int32MultiArray.html) with two integers: `[left_ticks, right_ticks]`.

### Node interface

**Publishes**

* Topic: `/wheel_ticks`
* Type: `std_msgs/msg/Int32MultiArray`
* `data[0] = left_ticks`, `data[1] = right_ticks`

**Subscribes:** none.

### 2.2 Parameters (recommended as ROS parameters)

* `publish_rate_hz` (float, 50.0)
* `ticks_per_rev` (int, 2048)
* `left_rps`, `right_rps` (float, any) — wheel speed in revolutions per second

### 2.3 Create a package and build

Create a Python package in your workspace:

```bash
cd ~/ros2_ws/src
ros2 pkg create --build-type ament_python --license Apache-2.0 --dependencies rclpy std_msgs <pkg_team_name>
```

Build the workspace:

```bash
cd ~/ros2_ws
colcon build
source install/setup.bash
```

### 2.4 Self-check

1. Implement new python code in your pkg
2. Set correct `entry_points` in `setup.py`/`setup.cfg`
3. Run ROS2 node:
```bash
ros2 run <pkg_team_name> <exec_name>
```

### 2.5 Verify output:

```bash
ros2 topic echo /wheel_ticks
```

---

## Task 3. Encoder driver

## 3.1 Goal

Implement a node `encoder_driver` that subscribes to ticks and publishes wheel angles as [`sensor_msgs/msg/JointState`](https://docs.ros.org/en/noetic/api/sensor_msgs/html/msg/JointState.html) on `/joint_states`. This is a common ROS 2 interface for joint positions.

### Node interface

**Subscribes**

* Topic: `/wheel_ticks`
* Type: `std_msgs/msg/Int32MultiArray`

**Publishes**

* Topic: `/joint_states`
* Type: `sensor_msgs/msg/JointState`

Recommended joint names:

* `left_wheel_joint`
* `right_wheel_joint`

### 3.2 Convert ticks to angle

<!-- Let `ticks_per_rev` be ticks per revolution. The wheel angle is:

$
\theta = 2\pi \cdot \frac{ticks}{ticks\_per\_rev}
$
-->
### 3.3 Self-check

1. Add new python code in your pkg
2. Don't forget to add `entry_point`
3. Run both your programs
4. Check result:
```bash
ros2 topic echo /joint_states
```



---

## Task 4. Odometry

### 4.1 Goal

Implement a node `wheel_odom` that computes differential-drive odometry from wheel rotation and publishes:

[ROS2 docs](https://docs.ros.org/en/humble/Tutorials/Intermediate/Tf2/Writing-A-Tf2-Broadcaster-Py.html)

* `/odom` ([`nav_msgs/msg/Odometry`](https://docs.ros.org/en/noetic/api/nav_msgs/html/msg/Odometry.html))
* TF transform `odom → base_link`

### Node interface

**Subscribes**

* `/joint_states` (`sensor_msgs/msg/JointState`)

**Publishes**

* `/odom` (`nav_msgs/msg/Odometry`)
* TF `odom → base_link` using `tf2_ros.TransformBroadcaster`

### 4.2 Robot parameters

* `wheel_radius` (m, 0.05)
* `wheel_base` (m, 0.30)

### 4.3 Differential drive kinematics
<!--
Let the update provide `Δθ_l`, `Δθ_r`.

$
\Delta s_l = r \cdot \Delta \theta_l,\quad \Delta s_r = r \cdot \Delta \theta_r
$

$
\Delta s = \frac{\Delta s_r + \Delta s_l}{2},\quad \Delta \psi = \frac{\Delta s_r - \Delta s_l}{b}
$

Pose update (midpoint integration):

$
x_{k+1} = x_k + \Delta s \cdot \cos(\psi_k + \Delta \psi/2)
$
$
y_{k+1} = y_k + \Delta s \cdot \sin(\psi_k + \Delta \psi/2)
$
$
\psi_{k+1} = \psi_k + \Delta \psi
$

Velocities:

$
v \approx \Delta s/\Delta t,\quad \omega \approx \Delta \psi/\Delta t
$
-->
### 4.4 Filling the Odometry message

* `header.frame_id` = "odom"
* `child_frame_id` = "base_link"
* `pose.pose.position.{x,y}` = (x,y)
* `pose.pose.orientation` = quaternion from yaw (`ψ`)
* `twist.twist.linear.x` = v
* `twist.twist.angular.z` = ω

### 4.5 TF verification

[Publish TF with tf2 (use the TransformBroadcaster pattern](https://docs.ros.org/en/humble/Tutorials/Intermediate/Tf2/Writing-A-Tf2-Broadcaster-Py.html)

Check:

```bash
ros2 run tf2_ros tf2_echo odom base_link
```

### 4.6 RViz2 visualization

* Start `rviz2`.
* Set Fixed Frame to `odom`.
* Add displays:

  * **TF**
  * **Odometry** (topic `/odom`) to visualize the path.

### 4.6 TF tree visualization

ros2 run 
> Checkpoint (for the report): 
> - RViz2 screenshot showing the TF tree and odometry visualization
> - TF tree screenshot
> - rqt_graph screenshot
